# Adversarial Attacks

References: 
* parts of the code below are based on https://pytorch.org/tutorials/beginner/fgsm_tutorial.html

### Imports

In [1]:
%matplotlib inline
%autosave 60

### Imports ###
import json
import sys
import random
import pandas as pd
import numpy as np
import pickle

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torchaudio

# Audio Player
import IPython.display as ipd

# Other Source Code
import utils

Autosaving every 60 seconds


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Loading Pre-Processed Data

In [3]:
X_train = torch.load("./cache/X_train.pt")
y_train = torch.load("./cache/y_train.pt")
paths_train = torch.load("./cache/paths_train.pt")
X_val = torch.load("./cache/X_val.pt")
y_val = torch.load("./cache/y_val.pt")
paths_val = torch.load("./cache/paths_val.pt")

print(len(X_train))
print(X_train[0].shape)
print(len(X_val))
print(X_val[0].shape)

5060
torch.Size([1, 80000])
1687
torch.Size([1, 80000])


### Load Model

In [4]:
model = utils.M5()
model.load_state_dict(torch.load("./best_model"))
model.to(device)

M5(
  (conv1): Conv1d(1, 128, kernel_size=(80,), stride=(4,))
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(256, 512, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (avgPool): AvgPool1d(kernel_size=(77,), stride=(77,), p

In [5]:
class EmergencyDataset(Dataset):
    
    def __init__(self, X, y, paths):

        self.X = X
        self.y = y
        self.paths = paths
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return len(self.X)
    
    def getPath(self, index):
        return self.paths[index]
    
train_set = EmergencyDataset(X_train, y_train, paths_train)
val_set = EmergencyDataset(X_val, y_val, paths_val)

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 64, shuffle = True, **kwargs)
val_loader = torch.utils.data.DataLoader(val_set, batch_size = 64, shuffle = True, **kwargs)

In [6]:
# sanity check - ensure all have the same shape after data augmentation
for i in range(len(X_val)):
    if(len(X_val[i].shape) != 2):
        print(X_val[i].shape)

for i in range(len(val_set)):
    if (len(val_set[i][0].shape) != 2): 
        print(val_set[i][0].shape)

### Evaluate Current Performance of the Model

In [7]:
def computePerformanceMetrics(mdl):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    mdl.eval()
    correct = 0
    
    for data, target in val_loader:
        data = data.to(device)
        target = target.to(device)
        output = mdl(data)
        output = output.permute(1, 0, 2)
        pred = output.max(2)[1] # get the index of the max log-probability
        correct += pred.eq(target).cpu().sum().item()
        
        with torch.no_grad():
            tp += torch.sum(pred & target)
            tn += torch.sum((pred == 0) & (target == 0))
            fp += torch.sum(((pred == 1) & (target == 0)))
            fn += torch.sum((target == 1) & (pred == 0))
            
    fp = fp.data.cpu().numpy()
    tp = tp.data.cpu().numpy()
    fn = fn.data.cpu().numpy()
    tn = tn.data.cpu().numpy()
    
    acc = (tp + tn) / (fp + fn + tp + tn)
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)
    f1 = 2*(prec*rec)/(prec+rec)
    
    print('Prec={:.2f}'.format(prec))
    print('Rec={:.2f}'.format(rec))
    print('F1={:.2f}'.format(f1))
    print("ACC={:.2f}".format(acc))

    print('\nVAL-ACC: {}/{} ({:.0f}%)\n'.format(correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    
computePerformanceMetrics(model)

Prec=0.87
Rec=0.85
F1=0.86
ACC=0.86

VAL-ACC: 1457/1687 (86%)



### Util

In [8]:
def softmax(a, b):
    res = np.exp(a) / (np.exp(a) + np.exp(b))
    if b > a: res = 1 - res
    return res

# FGSM

In [9]:
# FGSM attack code
def fgsm_attack(sample, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_sample = sample + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    # perturbed_isample = torch.clamp(perturbed_sample, 0, 1) # <- not necessary for audio!
    # Return the perturbed image
    return perturbed_sample

## Exploration of Attacking a given Sample

In [10]:
def predict(sample):
    
    # resize sample to fit model:                #sample.shape            = torch.Size([1, 80000])
    sample_unsqueezed = sample.unsqueeze(0)      #sample_unsqueezed.shape = torch.Size([1, 1, 80000])
    
    # put to gpu
    sample_cuda = sample_unsqueezed.to(device)
    
    # run model to get output scores
    output = model(sample_cuda)
    
    pred = output.max(2)[1]
    
    # compute softmax
    output_np = output.cpu().detach().numpy()
    a, b = output_np[0,0,0], output_np[0,0,1]
    em_prob = np.exp(b) / (np.exp(a) + np.exp(b))
    
    pred_value = pred.item()
    
    return pred_value, em_prob

def getRandomSample():
    N_val = len(val_set)
    random_index = random.randint(0, N_val - 1)
    random_sample, random_label = val_set[random_index]
    path = val_set.getPath(random_index)
    #print(random_sample.dtype)
    #print(random_sample.size)
    return random_sample, random_label, path

def adversarialAttack(sample, label, eps):
        
    # resize sample to fit model:                #sample.shape            = torch.Size([1, 80000])
    sample_unsqueezed = sample.unsqueeze(0)      #sample_unsqueezed.shape = torch.Size([1, 1, 80000])
    
    # put to gpu
    sample_cuda = sample_unsqueezed.to(device)
    
    # for FGSM we need grads w.r.t. sample!
    sample_cuda.requires_grad = True

    # run model to get output scores
    output = model(sample_cuda) 
    
    # make label to tensor in order to compute loss & move to gpu
    label_t = torch.tensor([label], dtype=torch.long, device=device)
    #print(label_tc) print(label_tc.shape) print(label_tc.dtype)

    # compute loss
    loss = F.nll_loss(output[0], label_t) 

    # Zero all existing gradients
    model.zero_grad()

    # Calculate gradients of model in backward pass
    loss.backward()

    # get grad
    data_grad = sample_cuda.grad.data

    # Call FGSM Attack
    perturbed_sample = fgsm_attack(sample_cuda, eps, data_grad)
    
    # squeeze back in original shape
    perturbed_sample_squeezed = perturbed_sample.squeeze(0)

    return perturbed_sample_squeezed

eps = 0.01  
random_sample, random_label, random_path = getRandomSample()
perturbed_sample = adversarialAttack(random_sample, random_label, eps)



print("Original Sample")
print(random_path)
print("Label: {}".format(random_label))
ipd.display(ipd.Audio(random_path))


print("Preprocessed Sample (Downsampled to 8K)")
prediction_downsampled = predict(random_sample)
print("Prediction: {} - EM-Prob = {:.2f}%".format(prediction_downsampled[0], prediction_downsampled[1]))
ipd.display(ipd.Audio(random_sample.detach().cpu(), rate=8000))

print("Adversarial Sample [with Epsilon={}]".format(eps))
prediction_adversarial = predict(perturbed_sample)
print("Prediction: {} - EM-Prob = {:.2f}%".format(prediction_adversarial[0], prediction_adversarial[1]))
ipd.display(ipd.Audio(perturbed_sample.detach().cpu(), rate=8000))

Original Sample
/nfs/students/summer-term-2020/project-4/data/dataset1/download/training_unbalanced/negative/krtiUKiUgN4.wav
Label: 0


Preprocessed Sample (Downsampled to 8K)
Prediction: 0 - EM-Prob = 0.17%


Adversarial Sample [with Epsilon=0.01]
Prediction: 1 - EM-Prob = 0.96%


## Attacking the whole Dataset

In [11]:
def adv_test( model, device, test_loader, epsilon ):

    # Accuracy counter
    correct = 0
    adv_examples = []

    # Loop over all examples in test set
    for data, target in test_loader:
                
        # Send the data and label to the device
        data, target = data.to(device), target.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True
        
        # Forward pass the data through the model
        output = model(data)
               
        # get original prediction 
        init_pred = output.max(2)[1]
        #init_pred = output.max(1, keepdim=True)[1] # change here for lager batch sizes!
        
        # If the initial prediction is wrong, dont bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        # Calculate the loss
        loss = F.nll_loss(output[0], target) #the loss functions expects a batchSizex10 input
        #loss = F.nll_loss(output, target) # change here for lager batch sizes!

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect datagrad
        data_grad = data.grad.data

        # Call FGSM Attack
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        #final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        final_pred = output.max(2)[1]


        if final_pred.item() == target.item():
            correct += 1
            # Special case for saving 0 epsilon examples
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:
            # Save some adv examples for visualization later
            if len(adv_examples) < 7:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_loader), final_acc))

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples


attack_loader = torch.utils.data.DataLoader(val_set, batch_size = 1, shuffle = True)

accuracies = []
examples = []
epsilons = [0, 0.01, 0.02, 0.03, 0.05, 0.1, 0.99, 10]
# Run test for each epsilon
for eps in epsilons:
    acc, ex = adv_test(model, device, attack_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

Epsilon: 0	Test Accuracy = 1457 / 1687 = 0.8636633076467102
Epsilon: 0.01	Test Accuracy = 368 / 1687 = 0.21813870776526378
Epsilon: 0.02	Test Accuracy = 282 / 1687 = 0.16716064018968582
Epsilon: 0.03	Test Accuracy = 315 / 1687 = 0.18672199170124482
Epsilon: 0.05	Test Accuracy = 380 / 1687 = 0.22525192649673978
Epsilon: 0.1	Test Accuracy = 504 / 1687 = 0.2987551867219917
Epsilon: 0.99	Test Accuracy = 769 / 1687 = 0.45583876704208653
Epsilon: 10	Test Accuracy = 781 / 1687 = 0.46295198577356256
